# Agentic Hotel Reservation & Travel Assistant

This notebook implements a **stateful, autonomous, agentic system** using:

- LangGraph (graph-based agent orchestration)
- Grok LLM (LLM reasoning)
- SQLite (persistent long-term + short-term memory)
- Session ID + User ID separation
- Guardrails embedded inside the graph
- External tools for tourism search
- Profile deletion & GDPR-style cleanup
- DOT graph visualization

Execution environment: Google Colab


In [1]:
!pip install langgraph langchain duckduckgo-search sqlite-utils pydot
!pip install -q groq langgraph langchain pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install ddgs
!pip install -q grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00


In [3]:
import os
import sqlite3
import uuid
import datetime
from typing import TypedDict, Optional, List

from groq import Groq
from langgraph.graph import StateGraph, END


In [4]:
os.environ["GROQ_API_KEY"] = ""
llm_client = Groq()


In [5]:
#CELL 4 — SQLite Database Initialization
conn = sqlite3.connect("travel_system.db", check_same_thread=False)
cursor = conn.cursor()


In [6]:
#CELL 5 — Create Database Tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id TEXT PRIMARY KEY,
    name TEXT,
    created_at TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS sessions (
    session_id TEXT,
    user_id TEXT,
    created_at TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS memory (
    user_id TEXT,
    role TEXT,
    content TEXT,
    timestamp TEXT
)
""")

conn.commit()


In [7]:
#CELL 6 — Insert 10 Dummy Users (MANDATORY)
dummy_users = [
    ("u1", "Alice"), ("u2", "Bob"), ("u3", "Carol"),
    ("u4", "David"), ("u5", "Eva"), ("u6", "Frank"),
    ("u7", "Grace"), ("u8", "Henry"), ("u9", "Ivy"), ("u10", "Jack")
]

for uid, name in dummy_users:
    cursor.execute(
        "INSERT OR IGNORE INTO users VALUES (?, ?, ?)",
        (uid, name, datetime.datetime.utcnow().isoformat())
    )

conn.commit()


/tmp/ipython-input-1653424759.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (uid, name, datetime.datetime.utcnow().isoformat())


In [8]:
#CELL 7 — Session & Memory Utilities
def create_session(user_id: str) -> str:
    sid = str(uuid.uuid4())
    cursor.execute(
        "INSERT INTO sessions VALUES (?, ?, ?)",
        (sid, user_id, datetime.datetime.utcnow().isoformat())
    )
    conn.commit()
    return sid


def save_memory(user_id: str, role: str, content: str):
    cursor.execute(
        "INSERT INTO memory VALUES (?, ?, ?, ?)",
        (user_id, role, content, datetime.datetime.utcnow().isoformat())
    )
    conn.commit()


def load_short_term_memory(user_id: str, limit: int = 10):
    cursor.execute("""
        SELECT role, content FROM memory
        WHERE user_id=?
        ORDER BY timestamp DESC
        LIMIT ?
    """, (user_id, limit))
    return cursor.fetchall()


In [9]:
#CELL 8 — Delete Profile (Hard Delete)
def delete_profile(user_id: str):
    cursor.execute("DELETE FROM users WHERE user_id=?", (user_id,))
    cursor.execute("DELETE FROM sessions WHERE user_id=?", (user_id,))
    cursor.execute("DELETE FROM memory WHERE user_id=?", (user_id,))
    conn.commit()


In [10]:
#CELL 9 — LLM Call Wrapper (Groq)
def call_llm(system_prompt: str, user_prompt: str) -> str:
    response = llm_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content


In [11]:
#CELL 10 — Guardrails (Anti-Hallucination)
def guardrail_check(text: Optional[str]) -> str:
    if text is None or len(text.strip()) < 15:
        return "I’m not trained on this information yet."
    return text


In [12]:
#CELL 11 — LangGraph State Definition
class GraphState(TypedDict):
    user_id: str
    session_id: str
    query: str
    route: Optional[str]
    response: Optional[str]
    iterations: int


In [13]:
#CELL 12 — Router Node
def router_node(state: GraphState):
    q = state["query"].lower()
    if "hotel" in q:
        state["route"] = "hotel"
    elif "restaurant" in q or "food" in q:
        state["route"] = "restaurant"
    elif "attraction" in q or "place" in q:
        state["route"] = "attraction"
    elif "delete" in q:
        state["route"] = "delete"
    else:
        state["route"] = "unknown"

    state["iterations"] += 1
    return state


In [14]:
#CELL 13 — Hotel Agent Node
def hotel_agent_node(state: GraphState):
    save_memory(state["user_id"], "user", state["query"])

    reply = call_llm(
        "You are a hotel recommendation assistant.",
        state["query"]
    )

    save_memory(state["user_id"], "assistant", reply)
    state["response"] = reply
    state["iterations"] += 1
    return state


In [15]:
#CELL 14 — Restaurant Agent Node
def restaurant_agent_node(state: GraphState):
    reply = call_llm(
        "You are a restaurant recommendation assistant.",
        state["query"]
    )
    state["response"] = reply
    state["iterations"] += 1
    return state


In [16]:
#CELL 15 — Attraction Agent Node
def attraction_agent_node(state: GraphState):
    reply = call_llm(
        "You are a tourism and attractions assistant.",
        state["query"]
    )
    state["response"] = reply
    state["iterations"] += 1
    return state


In [17]:
#CELL 16 — Delete Agent Node
def delete_agent_node(state: GraphState):
    delete_profile(state["user_id"])
    state["response"] = "Your profile and all memory have been deleted."
    state["iterations"] += 1
    return state


In [18]:
#CELL 17 — Unknown Handler
def unknown_node(state: GraphState):
    state["response"] = "I’m not trained on this information yet."
    state["iterations"] += 1
    return state


In [19]:
#CELL 18 — Guardrail Node
def guardrail_node(state: GraphState):
    state["response"] = guardrail_check(state["response"])
    state["iterations"] += 1
    return state


In [20]:
#CELL 19 — Summarizer Node
def summarizer_node(state: GraphState):
    return state


In [21]:
#CELL 20 — Build LangGraph
graph = StateGraph(GraphState)

graph.add_node("router", router_node)
graph.add_node("hotel", hotel_agent_node)
graph.add_node("restaurant", restaurant_agent_node)
graph.add_node("attraction", attraction_agent_node)
graph.add_node("delete", delete_agent_node)
graph.add_node("unknown", unknown_node)
graph.add_node("guardrail", guardrail_node)
graph.add_node("summarizer", summarizer_node)

graph.set_entry_point("router")

graph.add_conditional_edges(
    "router",
    lambda s: s["route"],
    {
        "hotel": "hotel",
        "restaurant": "restaurant",
        "attraction": "attraction",
        "delete": "delete",
        "unknown": "unknown"
    }
)

for node in ["hotel", "restaurant", "attraction", "delete", "unknown"]:
    graph.add_edge(node, "guardrail")

graph.add_edge("guardrail", "summarizer")
graph.add_edge("summarizer", END)

app = graph.compile()


In [22]:
#!pip install -q grandalf
_HAS_GRANDALF = False



In [23]:
print(app.get_graph().draw_ascii())


                                         +-----------+                                          
                                         | __start__ |                                          
                                         +-----------+                                          
                                                *                                               
                                                *                                               
                                                *                                               
                                           +--------+                                           
                                        ...| router |....                                       
                                 ..........+--------+... ........                               
                         ........   ....        .       ....     ........                       
                  .......     

In [25]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

# -------------------
# Register Nodes
# -------------------
graph.add_node("router", router_node)
graph.add_node("hotel", hotel_agent_node)
graph.add_node("restaurant", restaurant_agent_node)
graph.add_node("attraction", attraction_agent_node)
graph.add_node("delete", delete_agent_node)
graph.add_node("unknown", unknown_node)
graph.add_node("guardrail", guardrail_node)
graph.add_node("summarizer", summarizer_node)

# -------------------
# Entry Point
# -------------------
graph.set_entry_point("router")

# -------------------
# Conditional Routing
# -------------------
graph.add_conditional_edges(
    "router",
    lambda state: state["route"],
    {
        "hotel": "hotel",
        "restaurant": "restaurant",
        "attraction": "attraction",
        "delete": "delete",
        "unknown": "unknown"
    }
)

# -------------------
# Enforce Guardrails
# -------------------
for node in ["hotel", "restaurant", "attraction", "delete", "unknown"]:
    graph.add_edge(node, "guardrail")

# -------------------
# Finalization
# -------------------
graph.add_edge("guardrail", "summarizer")
graph.add_edge("summarizer", END)

# -------------------
# Compile Graph
# -------------------
app = graph.compile()


In [26]:
print(app.get_graph().draw_ascii())


                                         +-----------+                                          
                                         | __start__ |                                          
                                         +-----------+                                          
                                                *                                               
                                                *                                               
                                                *                                               
                                           +--------+                                           
                                        ...| router |....                                       
                                 ..........+--------+... ........                               
                         ........   ....        .       ....     ........                       
                  .......     

In [28]:
state = {
    "user_id": "u1",
    "session_id": create_session("u1"),
    "query": "Suggest a good hotel in Paris",
    "route": None,
    "response": None,
    "iterations": 0
}

result = app.invoke(state)

print("Final Response:", result["response"])
print("Iterations:", result["iterations"])


/tmp/ipython-input-1384976461.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (sid, user_id, datetime.datetime.utcnow().isoformat())
/tmp/ipython-input-1384976461.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (user_id, role, content, datetime.datetime.utcnow().isoformat())


Final Response: Paris, the City of Light! There are countless amazing hotels to choose from, but I'd like to recommend a few top-notch options. Here are a few suggestions:

1. **Hotel Plaza Athenee**: This 5-star hotel is located in the heart of Paris, on the prestigious Avenue Montaigne. It offers luxurious rooms, a world-class spa, and an exceptional dining experience. The hotel's elegant design and impeccable service make it a favorite among celebrities and world leaders.

Price range: Around €1,000-€2,000 per night

2. **Hotel Le Bristol**: This 5-star hotel is situated in the 8th arrondissement, near the Eiffel Tower. It boasts stunning views of the city, a beautiful garden, and an impressive art collection. The hotel's rooms are decorated with elegant furnishings and offer a high level of comfort.

Price range: Around €800-€1,800 per night

3. **Hotel Le Meurice**: This 5-star hotel is located on the Rue de Rivoli, near the Louvre Museum. It offers luxurious rooms, a world-class 

In [29]:
while True:
    q = input("User: ")

    state = {
        "user_id": "u1",
        "session_id": state["session_id"],
        "query": q,
        "route": None,
        "response": None,
        "iterations": 0
    }

    result = app.invoke(state)

    print("Agent Response:", result["response"])
    print("Iterations:", result["iterations"])

    if q.lower() == "exit":
        break


User: Suggest good hotel in india newdelhi


/tmp/ipython-input-1384976461.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (user_id, role, content, datetime.datetime.utcnow().isoformat())


Agent Response: New Delhi is a vibrant city with a rich history and culture. Here are some top-rated hotel suggestions in New Delhi:

**Luxury Hotels**

1. **The Oberoi New Delhi**: A 5-star hotel located in the heart of the city, offering luxurious rooms and suites, a spa, and fine dining options.
2. **The Taj Palace Hotel**: A 5-star hotel situated in the diplomatic enclave, offering elegant rooms, a spa, and multiple dining options.
3. **The Leela Palace New Delhi**: A 5-star hotel located in the heart of the city, offering luxurious rooms and suites, a spa, and fine dining options.

**Boutique Hotels**

1. **The Roseate New Delhi**: A 5-star boutique hotel located in the heart of the city, offering stylish rooms and suites, a spa, and fine dining options.
2. **The Lalit New Delhi**: A 5-star boutique hotel situated in the heart of the city, offering luxurious rooms and suites, a spa, and fine dining options.
3. **The Manor**: A 5-star boutique hotel located in the heart of the city